In [1]:
import pandas as pd
import numpy as np
from jupyter_utils import jupyter_setup
jupyter_setup()
import os
import librosa
import sounddevice as sd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python


In [4]:
metadata_dicova = pd.read_csv(r"data\Coswara_processed\DiCova\SecondDiCOVA_detailed_test_metedata.csv")
full_metadata = pd.read_csv(r"data\Coswara_processed\full_meta_data.csv")
original_metadata = pd.read_csv(r"data\Coswara_processed\original_metadata.csv")

In [5]:
metadata_dicova.rename(columns={'GENDER': 'gender'}, inplace=True)
metadata_dicova.columns

Index(['BREATHING_ID', 'COUGH_ID', 'SPEECH_ID', 'FUSION_ID', 'COVID_STATUS',
       'gender', 'age', 'covid_type', 'non_covid_healthy',
       'non_covid_resp_ail', 'non_covid_but_covid_like_symp'],
      dtype='object')

In [525]:
metadata_dicova

,BREATHING_ID,COUGH_ID,SPEECH_ID,FUSION_ID,COVID_STATUS,gender,age,covid_type,non_covid_healthy,non_covid_resp_ail,non_covid_but_covid_like_symp
0,EKANDFRP,ZKCHFUEP,VXCAIMWY,ASRQDLRT,n,m,32,none,True,False,False
1,QXIDXPCN,YCLAPIMT,IXXPUQJC,BEEPLUGK,n,m,25,none,True,False,False
2,RJNHLHRD,TKQPLXLE,XWUBOAIT,CVUPFQCZ,n,m,27,none,True,False,False
3,ACACIEDW,FJGMDZFI,DBXGCVUQ,YSDBVPHH,n,m,16,none,True,False,False
4,JVPUNCRM,PLKOEBZH,SBPTSFCM,JXGSKQEJ,n,m,34,none,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
466,BSELLAXS,BGAGCNVD,IGDXPHCU,FTAWSOYC,p,f,24,asymp,False,False,False
467,FALWRMJU,NVFVABXB,NMRCPJGL,FZBMXRWK,p,f,20,asymp,False,False,False
468,LHEEKCTA,RUMOJWCC,CUNGTJBJ,KLSRUVMB,p,f,20,mild,False,False,False
469,VELLTXHS,LIVFZTVY,MBDIWSZK,PMRMEAAT,p,f,23,mild,False,False,False


In [6]:
class AudioData:
    def __init__(self, file_path, user_id, age, gender):
        audio, self.sr = librosa.load(file_path, sr=None)
        # audio, self.sr = librosa.load(file_path, sr=44100)
        # self.id = os.path.splitext(os.path.basename(full_path))[0]
        self.file_path= file_path
        self.id = user_id
        self.audio_len = len(audio) * (44100/self.sr)
        self.target_id = []
        self.gender = gender
        self.age =age
        self.correlations = {}
        self.best_fit = None
        
    def __repr__(self):
        return f"id:{self.id} | len:{self.audio_len} | {self.gender}{self.age}"
            
    def __eq__(self, other):
        if isinstance(other, AudioData):
            return (
                self.audio_len == other.audio_len
                and self.gender == other.gender
                and self.age == other.age
            )
        return False        
    
    def eq_age_gender(self, other):
        if isinstance(other, AudioData):
            return (self.gender == other.gender and self.age == other.age)
        return False  
    
    def comapre_audio(self, other):
        try:
            audio_orig, _ = librosa.load(self.file_path, sr=44100)
            audio_comp, _ = librosa.load(other.file_path, sr=44100)
            audio_orig, audio_comp = librosa.util.normalize(audio_orig), librosa.util.normalize(audio_comp)
        except:
            return 0
        segment_len = 10000 if len(audio_orig) > 10000 else len(audio_orig)
        cross_corr = np.correlate(audio_orig[:10000], audio_comp, mode='full')
        return cross_corr.max()
        
    def get_audio(self):
        audio_orig, _ = librosa.load(self.file_path, sr=44100)
        return librosa.util.normalize(audio_orig)

In [7]:
# for full_id in list(full_metadata["user_id"])
dicova_audio_data = []
base_path = r"data\Coswara_processed\DiCova\Second_DiCOVA_Challenge_Test_Data_Release\AUDIO\breathing"
for breath_id in tqdm(list(metadata_dicova["BREATHING_ID"])):
    age = metadata_dicova[metadata_dicova["BREATHING_ID"] == breath_id]["age"].item()
    gender = metadata_dicova[metadata_dicova["BREATHING_ID"] == breath_id]["gender"].item()[0]
    full_path = f"{os.path.join(base_path, breath_id)}.flac"
    dicova_audio_data.append(AudioData(full_path, breath_id, age, gender))
    # print(full_path, age, gender)

  0%|          | 0/471 [00:00<?, ?it/s]

In [8]:
my_audio_data = []
base_path = r"data\Coswara_processed\Recordings"
for user_id in tqdm(list(full_metadata["user_id"])):
    age = full_metadata[full_metadata["user_id"] == user_id]["age"].item()
    gender = full_metadata[full_metadata["user_id"] == user_id]["gender"].item()[0]
    full_path = f"{os.path.join(base_path, user_id, 'breathing-deep')}.wav"
    my_audio_data.append(AudioData(full_path, user_id, age, gender))
    # print(full_path, age, gender)

  0%|          | 0/2746 [00:00<?, ?it/s]

In [9]:
# base_path = r"data\Coswara_processed\Recordings"
# user_id = full_metadata["user_id"][1]
# full_path = f"{os.path.join(base_path, user_id, 'breathing-deep')}.wav"
# age = full_metadata[full_metadata["user_id"] == user_id]["age"].item()
# gender = full_metadata[full_metadata["user_id"] == user_id]["gender"].item()[0]
# audio_data1 = AudioData(full_path, user_id, age, gender)


# base_path = r"data\Coswara_processed\DiCova\Second_DiCOVA_Challenge_Test_Data_Release\AUDIO\breathing"
# breath_id = metadata_dicova["BREATHING_ID"][0]
# full_path = f"{os.path.join(base_path, breath_id)}.flac"
# age = metadata_dicova[metadata_dicova["BREATHING_ID"] == breath_id]["age"].item()
# gender = metadata_dicova[metadata_dicova["BREATHING_ID"] == breath_id]["gender"].item()[0]
# audio_data2 = AudioData(full_path, breath_id, age, gender)
# audio_data2, audio_data1

In [10]:
for dicova_id in dicova_audio_data:
    # print("############################################")
    dicova_id.target_id = []
    for my_id in my_audio_data:
        if dicova_id.eq_age_gender(my_id):
        # if dicova_id == my_id:
        # if np.abs(dicova_id.audio_len - my_id.audio_len) < 100:
            diff = dicova_id.audio_len - my_id.audio_len
            diff_percent = diff / dicova_id.audio_len * 100
            diff_percent = np.round(diff_percent)
            if diff_percent < 0 and diff_percent > -25:
                # print(diff_percent)
                dicova_id.target_id.append(my_id)

In [ ]:
for dicova_id in dicova_audio_data:
    print(len(dicova_id.target_id))
    # print(dicova_id.sr)

In [ ]:
for sample in tqdm(dicova_audio_data):
    max_corr = 0
    best_fit = None
    for user_id in sample.target_id:
        corr = sample.comapre_audio(user_id)
        sample.correlations[user_id.id] = corr
        print(corr)
        if corr > max_corr:
            max_corr = corr
            best_fit = user_id
    sample.best_fit = best_fit

In [30]:
actual_fits = {}

In [34]:
idx = 0

In [ ]:
idx += 1
print(idx)
target_audio = dicova_audio_data[idx].best_fit.get_audio()
source_audio = dicova_audio_data[idx].get_audio()
plt.plot(target_audio)
plt.figure()
plt.plot(source_audio)

301


MemoryError: Unable to allocate 9.55 MiB for an array with shape (626062, 2) and data type float64

In [1]:
actual_fits[dicova_audio_data[idx].id] = dicova_audio_data[idx].best_fit.id
actual_fits

NameError: name 'dicova_audio_data' is not defined

In [407]:
len(actual_fits)

138

In [442]:
import csv

In [458]:
df = pd.DataFrame({"breath ID": actual_fits.keys(), "user ID":actual_fits.values()})

In [461]:
df.to_csv("temporary_id_relationshipts.csv", index=False)